In [58]:
import pandas as pd

In [59]:
# Replace with the actual path to your file
emp = pd.read_csv("C:/Users/DELL360/Downloads/Employee.csv")

# Quick preview
print(emp.head())
print(emp.info())

   Education  JoiningYear       City  PaymentTier  Age  Gender EverBenched  \
0  Bachelors         2017  Bangalore            3   34    Male          No   
1  Bachelors         2013       Pune            1   28  Female          No   
2  Bachelors         2014  New Delhi            3   38  Female          No   
3    Masters         2016  Bangalore            3   27    Male          No   
4    Masters         2017       Pune            3   24    Male         Yes   

   ExperienceInCurrentDomain  LeaveOrNot  
0                          0           0  
1                          3           1  
2                          2           0  
3                          5           1  
4                          2           1  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4653 entries, 0 to 4652
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Education                  4653 non-null   object
 1

In [60]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from imblearn.over_sampling import SMOTE
from scipy.stats import zscore
import numpy as np

# 1. Load your dataset
emp = pd.read_csv("C:/Users/DELL360/Downloads/Employee.csv")

# 2. Handle missing values (if any)
emo = emp.dropna()

# 3. Encode categorical variables
categorical_cols = ['Education', 'City', 'Gender', 'EverBenched']
emp_encoded = pd.get_dummies(emp, columns=categorical_cols, drop_first=True, dtype= int)

# 4. Feature and target split
X = emp_encoded.drop('LeaveOrNot', axis=1)
y = emp_encoded['LeaveOrNot']

# 5. Train-test split
# stratify=y is used to splitting the traing data is appx. in equal 
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=42)

# 6. Train logistic regression
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)

# 7. Predict and evaluate
y_pred = lr.predict(X_test)
print("🔹 Accuracy:", accuracy_score(y_test, y_pred))
print("\n🔹 Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


# Precision
print("🔹 Precision:", precision_score(y_test, y_pred))

# Recall
print("🔹 Recall:", recall_score(y_test, y_pred))

# F1 Score
print("🔹 F1 Score:", f1_score(y_test, y_pred))




🔹 Accuracy: 0.7428366762177651

🔹 Confusion Matrix:
 [[816 100]
 [259 221]]
🔹 Precision: 0.6884735202492211
🔹 Recall: 0.46041666666666664
🔹 F1 Score: 0.5518102372034956


C:\Users\DELL360\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [61]:
emp_encoded.head()

,JoiningYear,PaymentTier,Age,ExperienceInCurrentDomain,LeaveOrNot,Education_Masters,Education_PHD,City_New Delhi,City_Pune,Gender_Male,EverBenched_Yes
0,2017,3,34,0,0,0,0,0,0,1,0
1,2013,1,28,3,1,0,0,0,1,0,0
2,2014,3,38,2,0,0,0,1,0,0,0
3,2016,3,27,5,1,1,0,0,0,1,0
4,2017,3,24,2,1,1,0,0,1,1,1


In [62]:
print(emp_encoded['Age'].min())
print(emp_encoded['Age'].max())

22
41


In [63]:
# 4. Apply IQR filtering on person_age only
Q1 = emp_encoded['Age'].quantile(0.25)
Q3 = emp_encoded['Age'].quantile(0.75)
IQR = Q3 - Q1
lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR
print("lower: ", lower)
print("upper: ", upper)

lower:  17.0
upper:  41.0


In [64]:
valid_age_rows = emp_encoded['Age'].between(lower, upper)
emp_filtered = emp_encoded[valid_age_rows]

print("Rows before age outlier removal:", emp_encoded.shape[0])
print("Rows after  age outlier removal:", emp_filtered.shape[0])

Rows before age outlier removal: 4653
Rows after  age outlier removal: 4653


In [65]:
# 5. Feature/target split
X = emp_filtered.drop('LeaveOrNot', axis=1)
y = emp_filtered['LeaveOrNot']

# 6. Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=42)

# 7. Logistic Regression
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)

# 8. Evaluate
y_pred = lr.predict(X_test)
print("\nAccuracy after removing outliers from 'person_age':", accuracy_score(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


Accuracy after removing outliers from 'person_age': 0.7428366762177651

Confusion Matrix:
 [[816 100]
 [259 221]]


C:\Users\DELL360\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [66]:
#SMOTE

In [67]:
emp_filtered.head()

,JoiningYear,PaymentTier,Age,ExperienceInCurrentDomain,LeaveOrNot,Education_Masters,Education_PHD,City_New Delhi,City_Pune,Gender_Male,EverBenched_Yes
0,2017,3,34,0,0,0,0,0,0,1,0
1,2013,1,28,3,1,0,0,0,1,0,0
2,2014,3,38,2,0,0,0,1,0,0,0
3,2016,3,27,5,1,1,0,0,0,1,0
4,2017,3,24,2,1,1,0,0,1,1,1


In [68]:
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


# 1. Split features and target after age outlier removal
X = emp_filtered.drop('LeaveOrNot', axis=1)
y = emp_filtered['LeaveOrNot']

print(y.head())

print("Class distribution before SMOTE:\n", y.value_counts())


0    0
1    1
2    0
3    1
4    1
Name: LeaveOrNot, dtype: int64
Class distribution before SMOTE:
 LeaveOrNot
0    3053
1    1600
Name: count, dtype: int64


In [69]:
# 2. Apply SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

print("Class distribution before SMOTE:", y.value_counts())
print("Class distribution after SMOTE:", y_resampled.value_counts())


Class distribution before SMOTE: LeaveOrNot
0    3053
1    1600
Name: count, dtype: int64
Class distribution after SMOTE: LeaveOrNot
0    3053
1    3053
Name: count, dtype: int64


In [70]:
# 3. Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)
#X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, stratify=y_resampled, test_size=0.3, random_state=42)
#print(y_train.value_counts())
#print(y_test.value_counts())

In [71]:

# 4. Train Logistic Regression
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)

# 5. Evaluate
y_pred = lr.predict(X_test)
print("Accuracy after SMOTE:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy after SMOTE: 0.6561135371179039
Confusion Matrix:
 [[654 269]
 [361 548]]


In [72]:
#DATA NORMALIZATION

In [73]:
print(X_resampled.head())

   JoiningYear  PaymentTier  Age  ExperienceInCurrentDomain  \
0         2017            3   34                          0   
1         2013            1   28                          3   
2         2014            3   38                          2   
3         2016            3   27                          5   
4         2017            3   24                          2   

   Education_Masters  Education_PHD  City_New Delhi  City_Pune  Gender_Male  \
0                  0              0               0          0            1   
1                  0              0               0          1            0   
2                  0              0               1          0            0   
3                  1              0               0          0            1   
4                  1              0               0          1            1   

   EverBenched_Yes  
0                0  
1                0  
2                0  
3                0  
4                1  


In [74]:
# 3. Normalize features using StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_resampled)
print(X_scaled.shape)
print(X_scaled)

(6106, 10)
[[ 0.96950122  0.60244495  0.97952969 ... -0.63332568  0.91484613
  -0.32888747]
 [-1.13961945 -2.84187049 -0.27509569 ...  1.57896644 -1.09307999
  -0.32888747]
 [-0.61233929  0.60244495  1.81594661 ... -0.63332568 -1.09307999
  -0.32888747]
 ...
 [-0.08505912  0.60244495  0.14311277 ... -0.63332568 -1.09307999
  -0.32888747]
 [-0.61233929 -2.84187049 -0.69330415 ...  1.57896644 -1.09307999
  -0.32888747]
 [-0.08505912 -1.11971277 -0.90240838 ... -0.63332568 -1.09307999
  -0.32888747]]


In [75]:
import numpy as np

# Option 2: Print as DataFrame for clarity
scaled_df = pd.DataFrame(X_scaled, columns=X.columns)
print(scaled_df)
means_df = scaled_df.mean()
print("\n Mean of each scaled feature:\n", means_df)
print("\n Mean of each scaled feature:\n", means_df.round())

# Mean of each column after Normalization is 0
sd_df = pd.DataFrame(X_scaled, columns=X.columns).std()
print("\n SD of each scaled feature:\n", sd_df)
print("\n SD of each scaled feature:\n", sd_df.round())


      JoiningYear  PaymentTier       Age  ExperienceInCurrentDomain  \
0        0.969501     0.602445  0.979530                  -1.835935   
1       -1.139619    -2.841870 -0.275096                   0.084719   
2       -0.612339     0.602445  1.815947                  -0.555499   
3        0.442221     0.602445 -0.484200                   1.365155   
4        0.969501     0.602445 -1.111513                  -0.555499   
...           ...          ...       ...                        ...   
6101    -0.612339    -1.119713 -1.111513                  -0.555499   
6102    -1.139619     0.602445 -0.902408                   0.084719   
6103    -0.085059     0.602445  0.143113                  -1.835935   
6104    -0.612339    -2.841870 -0.693304                   0.724937   
6105    -0.085059    -1.119713 -0.902408                   0.084719   

      Education_Masters  Education_PHD  City_New Delhi  City_Pune  \
0             -0.484286      -0.180665       -0.541889  -0.633326   
1        

In [76]:
# 4. Train-test split with stratify
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_resampled, test_size=0.3, random_state=42
)

# 5. Train Logistic Regression
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)

# 6. Evaluate
y_pred = lr.predict(X_test)
print("\n🔹 Accuracy after Normalization:", accuracy_score(y_test, y_pred))
print("\n🔹 Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


🔹 Accuracy after Normalization: 0.6839519650655022

🔹 Confusion Matrix:
 [[659 264]
 [315 594]]


In [77]:
#XGBOOST 

In [78]:
from xgboost import XGBClassifier

# 5. Train XGBoost Classifier (instead of Logistic Regression)
xgb = XGBClassifier(random_state=42)
xgb.fit(X_train, y_train)

# 6. Evaluate
y_pred = xgb.predict(X_test)
print("\n🔹 Accuracy with XGBoost:", accuracy_score(y_test, y_pred))
print("\n🔹 Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

ModuleNotFoundError: No module named 'xgboost'

In [ ]:
!jupyter nbconvert --to script employee.ipynb

In [ ]:
import pickle
import os

# Make sure model exists
print(lr)

with open("employee_model.pkl", "wb") as f:
    pickle.dump(lr, f)

print("File size:", os.path.getsize("employee_model.pkl"), "bytes")


In [ ]:
with open("nominal_encoder.pkl", "wb") as f:
    pickle.dump(lr, f)

import os
print("Encoder size:", os.path.getsize("nominal_encoder.pkl"), "bytes")


In [79]:
from sklearn.preprocessing import OneHotEncoder
import pickle

nominal_cols = ['Education', 'City', 'Gender', 'EverBenched']

ohe = OneHotEncoder(
    handle_unknown='ignore',
    sparse_output=False
)

ohe.fit(emp[categorical_cols])   # df = your training dataframe

with open("nominal_encoder.pkl", "wb") as f:
    pickle.dump(lr, f)
